### Bibliotecas

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import time
import pandas as pd
import re
from tqdm.auto import tqdm

In [2]:
# Função para determinar se houve alagamento ou não em um determinado dia
def houve_alag(soup):
    # Texto padrão quando não houve alagamentos
    texto = 'Não há registros de alagamentos para essa data.'

    # Variável para alagamento. 0, sem alagamento. 1, com alagamento
    flag_alag = 0

    alag = soup.find_all(string = re.compile('registros'))
    
    if (alag == []):
        flag_alag = 1
        return flag_alag
    else:
        flag_alag = 0
        return flag_alag

In [3]:
# Cria dataframe para receber o conteúdo
Campos = ['Data',
           'Alagamento',
           'Bairro',
           'Tipo',
           'Horário Inicial',
           'Horário Final',
           'Endereço',
           'Sentido',
           'Referência'
           ]
df = pd.DataFrame(columns=Campos)

# Montagem dinâmica das datas a serem varridas:
start = date(2015, 10, 1)
end = date(2016, 1, 1)

# Datas com problemas
# 13/01/2019
# 15/11/2019
# 22/06/2021
# 29/10/2022
# 12/03/2023
# 17/01/2017

# Quando a função estiver totalmente funcional
# end = date.today()
data_pesquisada = start
dia = timedelta(days=1)

In [4]:
index= 0

# Inicia o preenchimento do dataframe
while data_pesquisada < end:
    data_formatada = data_pesquisada.strftime('%d')+'%2F'+data_pesquisada.strftime('%m')+'%2F'+data_pesquisada.strftime('%Y')
    URL_pesquisada = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca='+data_formatada+'&enviaBusca=Buscar'
    #print(URL_pesquisada)

    # Faz a requisição para a página
    try:
        response = requests.get(URL_pesquisada)
    except requests.exceptions.Timeout:
        print('Timeout informado pelo sistema')
    except requests.exceptions.TooManyRedirects:
        print('Problema na formação da URL. Favor tentar com outra')
    except requests.exceptions.RequestException as e:
        print('Erro catastrófico')
        raise SystemExit(e)

    # Analisa o HTML da página com o BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
        
    if (houve_alag(soup)==0):
        print(f"Em {data_pesquisada} não houve registros de alagamentos.")
        #df.at[index,'Data'] =  data_pesquisada
        #df.at[index,'Alagamento'] = 0
        #df.at[index,'Bairro'] = 0
        #df.at[index,'Tipo'] = 0
        #df.at[index,'Horário Inicial'] = 0
        #df.at[index,'Horário Final'] = 0
        #df.at[index,'Endereço'] = 0
        #df.at[index,'Sentido'] = 0
        #df.at[index,'Referência'] = 0
        #index = index +1 
    else:
        index_linhas = 0
        for pontos in soup.find_all('td', {'class':'total-pts arial-bairros-alag'}):
            texto = pontos.string
            no_alag = int(texto[0:texto.find('pt')-1])
            print(f'Data: {data_pesquisada}, No. alagamentos: {no_alag}')

            linhas = soup.find_all('div',{'class':'ponto-de-alagamento'})
            

            # Vamos iterar sobre a quantidade de pontos de alagamento
            for pt_alag in range(no_alag):
            
                df.at[index,'Data'] =  data_pesquisada
                df.at[index,'Alagamento'] = 1

                pai = pontos.parent
            
                df.at[index,'Bairro'] = pai.find('td',{'class':'bairro arial-bairros-alag linha-pontilhada'}).text.strip()
                df.at[index,'Tipo'] = linhas[index_linhas].find('li')['title']
            
                #'Horário Inicial',
                texto1 = linhas[index_linhas].find('li', {'class':'arial-descr-alag col-local'}).text
                horas =re.findall('(?:(\d\d?):)?([0-5][0-9])',texto1)
                df.at[index,'Horário Inicial'] = str(horas[0][0])+':'+str(horas[0][1])
                
                # Tenta capturar o 'Horário Final', se não conseguir, preenche com "N/A"
                try:
                    df.at[index,'Horário Final'] = str(horas[1][0])+':'+str(horas[1][1])
                    # Usa o horário final para encontrar o início do endereço
                    pos_endereco = texto1.find(horas[1][1]) + 2
                except IndexError:
                    df.at[index,'Horário Final'] = "N/A"  
                    # Se não houver horário final, encontra o endereço baseado no horário inicial
                    pos_endereco = texto1.find(horas[0][1]) + 2
    
                # Extrai o endereço usando a posição determinada anteriormente
                df.at[index, 'Endereço'] = texto1[pos_endereco:].strip()

                #'Sentido',
                texto2 = linhas[index_linhas].find_all('li', {'class':'arial-descr-alag'})
                info = texto2[1].get_text()
                df.at[index,'Sentido'] = info[8:info.find('Ref')]

                #'Referência'
                df.at[index,'Referência'] = info[info.find('Ref')+11:len(info)]
            
                index = index +1 
                index_linhas = index_linhas +1
    data_pesquisada = data_pesquisada + dia
    time.sleep(2)

Em 2015-10-01 não houve registros de alagamentos.
Em 2015-10-02 não houve registros de alagamentos.
Data: 2015-10-03, No. alagamentos: 2
Data: 2015-10-03, No. alagamentos: 1
Data: 2015-10-03, No. alagamentos: 1
Data: 2015-10-03, No. alagamentos: 2
Data: 2015-10-03, No. alagamentos: 2
Em 2015-10-04 não houve registros de alagamentos.
Em 2015-10-05 não houve registros de alagamentos.
Em 2015-10-06 não houve registros de alagamentos.
Em 2015-10-07 não houve registros de alagamentos.
Em 2015-10-08 não houve registros de alagamentos.
Em 2015-10-09 não houve registros de alagamentos.
Em 2015-10-10 não houve registros de alagamentos.
Em 2015-10-11 não houve registros de alagamentos.
Em 2015-10-12 não houve registros de alagamentos.
Data: 2015-10-13, No. alagamentos: 1
Em 2015-10-14 não houve registros de alagamentos.
Em 2015-10-15 não houve registros de alagamentos.
Em 2015-10-16 não houve registros de alagamentos.
Em 2015-10-17 não houve registros de alagamentos.
Em 2015-10-18 não houve regi

In [5]:
df.to_csv('Alagamentos_SP_2015_Q4.csv', encoding='utf-8-sig')